In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import coremltools as ct
import sys
import os
sys.path.append('..')
from segment_anything import sam_model_registry

In [ ]:
checkpoint_dir = '/Users/anatoli/Documents/segment-anything/checkpoints'

model_type = 'vit_b' # 'vit_b', 'vit_l' or 'vit_h'
if model_type == 'vit_h':
    checkpoint = 'sam_vit_h_4b8939.pth'
elif model_type == 'vit_b':
    checkpoint = 'sam_vit_b_01ec64.pth'
elif model_type == 'vit_l':
    checkpoint = 'sam_vit_l_0b3195.pth'
else:
    raise ValueError(f'Unknown model type {model_type}, model type must be one of "vit_b", "vit_l", "vit_h"')


In [ ]:
from segment_anything.utils.coreml import SamEmbedder, SamPointDecoder

sam = sam_model_registry[model_type](checkpoint=os.path.join(checkpoint_dir, checkpoint))
point_decoder_model = SamPointDecoder(sam, return_single_mask=False)
image_embedder_model = SamEmbedder(sam)

embed_dim = sam.prompt_encoder.embed_dim
embed_size = sam.prompt_encoder.image_embedding_size
mask_input_size = [4 * x for x in embed_size]
point_decoder_dummy_inputs = {
    "image_embeddings": torch.randn(1, embed_dim, *embed_size, dtype=torch.float),
    "point_coords": torch.randint(low=0, high=1024, size=(1, 5, 2), dtype=torch.float),
    "point_labels": torch.randint(low=0, high=4, size=(1, 5), dtype=torch.float),
    "mask_input": torch.randn(1, 1, *mask_input_size, dtype=torch.float),
    "has_mask_input": torch.tensor([1], dtype=torch.float),
}
image_embedder_dummy_inputs = {
    # The image as a torch tensor in 3xHxW format, already transformed for input to the model.
    'image': torch.randint(low=0, high=255, size=(1, 3, 1024, 1024), dtype=torch.float),
}


In [ ]:
point_decoder_trace = torch.jit.trace(point_decoder_model.eval(), tuple(point_decoder_dummy_inputs.values()))
image_embedder_trace = torch.jit.trace(image_embedder_model.eval(), tuple(image_embedder_dummy_inputs.values()))

# Save the traced model
torch.jit.save(point_decoder_trace, f'point_decoder_{model_type}_model.pt')
torch.jit.save(image_embedder_trace, f'image_embedder_{model_type}_model.pt')

In [ ]:
point_decoder_coreml_inputs = {
    "image_embeddings": ct.TensorType(name="image_embeddings", shape=point_decoder_dummy_inputs["image_embeddings"].size()),
    "point_coords": ct.TensorType(name="point_coords", shape=point_decoder_dummy_inputs["point_coords"].size()),
    "point_labels": ct.TensorType(name="point_labels", shape=point_decoder_dummy_inputs["point_labels"].size()),
    "mask_input": ct.TensorType(name="mask_input", shape=point_decoder_dummy_inputs["mask_input"].size()),
    "has_mask_input": ct.TensorType(name="has_mask_input", shape=point_decoder_dummy_inputs["has_mask_input"].size()),
}
point_decoder_coreml_outputs = {
    "iou_predictions": ct.TensorType(name="iou_predictions"),
    "low_res_masks": ct.TensorType(name="low_res_masks")
}
point_decoder_coreml_model = ct.convert(
    point_decoder_trace,
    outputs=list(point_decoder_coreml_outputs.values()),
    inputs=list(point_decoder_coreml_inputs.values()),
    minimum_deployment_target=ct.target.iOS15
)

image_embedder_coreml_inputs = {
    'image': ct.ImageType(name='image', shape=image_embedder_dummy_inputs['image'].shape, channel_first=True),
}
image_embedder_coreml_outputs = {
    "image_embeddings": ct.TensorType(name="image_embeddings")
}
image_embedder_coreml_model = ct.convert(
    image_embedder_trace,
    outputs=list(image_embedder_coreml_outputs.values()),
    inputs=list(image_embedder_coreml_inputs.values()),
    minimum_deployment_target=ct.target.iOS15
)

In [ ]:
point_decoder_coreml_model.save(f"point_decoder_{model_type}.mlpackage")
image_embedder_coreml_model.save(f"image_embedder_{model_type}.mlpackage")

Everything below this point is for exporting the entire pytorch model (embeddings and all) directly using coreml, and not the embeddings-to-masks part via the onnx model.

The following cells allow you to test the model on a single image, without specifying prompts (points, bboxes).

In [ ]:
import cv2
from segment_anything import SamAutomaticMaskGenerator
image = cv2.imread('/Users/anatoli/Downloads/Untitled_anatoli.jpeg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(image)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


In [ ]:
for i, mask in enumerate(masks):
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.axis('off')
    show_mask(mask['segmentation'], plt.gca())
    plt.show()